In [ ]:
import requests
import shutil
import re

from bs4 import BeautifulSoup

rs = requests.session()
res = rs.get('http://gcis.nat.gov.tw/gps/gpsInit/GpsInit/query.do')
soup = BeautifulSoup(res.text, 'html.parser')


# Get name of keyword's image
imagename = soup.findAll('input')[8]['value']
soup.findAll('input')[8].encode('utf8')
print imagename

# Get tokenID
token = soup.findAll('input')[-2]['value'].encode('utf8')
print token

# Fetch the keyword's image and store as check.png
res = rs.get('http://gcis.nat.gov.tw/gps/gps/codeimage/%s' %(imagename), stream=True)
f = open('check.png', 'wb')
shutil.copyfileobj(res.raw, f)
f.close()

from pyocr import tesseract as tool
from PIL import Image

# Recognize 
imagecode = tool.image_to_string(
    Image.open('check.png'),
    builder=pyocr.builders.TextBuilder()
)
print "Result : %s" % imagecode

payload = {
    'selCmpyType':1,
    'selQueryType':1,
    'queryData':'統一超商',
    'imageCode':imagecode,
    'imageFileName':imagename,
    'otherEnterFlag':'false',
    'useEUC':'N',
    'isShowEUC':'Y',
    'firstPage':0,
    'previousPage':0,
    'nextPage':0,
    'lastPage':0,
    'struts.token.name':'struts.token',
    'struts.token':token
}

print payload

res = rs.post('http://gcis.nat.gov.tw/gps/gpsInit/GpsInit/query.do', data=payload)

print res.text